In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/test.ft.txt.bz2
/kaggle/input/amazonreviews/train.ft.txt.bz2


In [5]:
df=pd.read_csv("/kaggle/input/amazonreviews/train.ft.txt.bz2")

ParserError: Error tokenizing data. C error: Expected 3 fields in line 3, saw 8


In [7]:
import bz2

reviews = []
labels = []

path = "/kaggle/input/amazonreviews/train.ft.txt.bz2"

with bz2.open(path, 'rt') as f:
    for i, line in enumerate(f):
        if i > 50000:   # বড় dataset, তাই limit দিচ্ছি
            break
            
        label, text = line.split(' ', 1)
        labels.append(1 if label == '__label__2' else 0)
        reviews.append(text.strip())

print("Loaded:", len(reviews))


Loaded: 50001


In [8]:
import pandas as pd

df = pd.DataFrame({
    "review": reviews,
    "sentiment": labels
})

df.head()


,review,sentiment
0,Stuning even for the non-gamer: This sound tra...,1
1,The best soundtrack ever to anything.: I'm rea...,1
2,Amazing!: This soundtrack is my favorite music...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1


In [6]:
df.head()

NameError: name 'df' is not defined

In [9]:
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)     # remove links
    text = re.sub(r'[^a-z\s]', '', text)    # remove special characters
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df["review"] = df["review"].apply(clean_text)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    df["review"],
    df["sentiment"],
    test_size=0.2,
    random_state=42
)


In [16]:
vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english'
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [17]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000)

In [18]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8802119788021198

Classification Report:

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4963
           1       0.87      0.89      0.88      5038

    accuracy                           0.88     10001
   macro avg       0.88      0.88      0.88     10001
weighted avg       0.88      0.88      0.88     10001


Confusion Matrix:

[[4304  659]
 [ 539 4499]]
